# Implementação

In [1]:
import requests

URL = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

r = requests.get(url=URL)

In [1]:
companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2021
last_year = 2021

In [3]:
from datetime import datetime

if first_year and last_year:
    year_range = [x for x in range(first_year, last_year + 1)]
elif first_year:
    last_year_tmp = datetime.now().year
    year_range = [x for x in range(first_year, last_year_tmp + 1)]
else:
    first_year_tmp = 2010
    year_range = [x for x in range(first_year_tmp, last_year + 1)]


year_filter = [f"{year}.zip" for year in year_range]

year_filter

['2021.zip']

In [4]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, "html.parser")

files_list = [x.get("href") for x in html.find("pre").find_all("a") if x.get("href")[-8:] in year_filter]

files_list

['itr_cia_aberta_2021.zip']

In [5]:
full_links = [f"{URL}{file_}" for file_ in files_list]
full_links

['https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2021.zip']

In [6]:
import pandas as pd
import os
import zipfile
import io

from peers_comparison import config

TMP_FILES_PATH = "tmp_files/"

complete_df = pd.DataFrame()

for link in full_links:
    tmp_file = requests.get(link, stream=True)
    unzipped = zipfile.ZipFile(io.BytesIO(tmp_file.content))

    filtered_files = [file_ for file_ in unzipped.namelist() for file_type in type_docs if file_type in file_]

    for selected_file in filtered_files:
        print(selected_file)
        unzipped.extract(selected_file, path=TMP_FILES_PATH)

        df = pd.read_csv(f"{TMP_FILES_PATH}{selected_file}", sep=";", decimal=",", encoding="Latin1")
        df["VL_CONTA"] = df["VL_CONTA"].astype("float64")

        df = df[df["ORDEM_EXERC"] == config.DEFAULT_ORDEM_EXERC].reset_index(drop=True).copy()

        df["source_file"] = selected_file

        complete_df = pd.concat([complete_df, df])

        os.remove(f"{TMP_FILES_PATH}{selected_file}")

os.rmdir(TMP_FILES_PATH)
# 1/0

itr_cia_aberta_BPA_con_2021.csv
itr_cia_aberta_BPA_ind_2021.csv
itr_cia_aberta_BPP_con_2021.csv
itr_cia_aberta_BPP_ind_2021.csv
itr_cia_aberta_DFC_MD_con_2021.csv
itr_cia_aberta_DFC_MD_ind_2021.csv
itr_cia_aberta_DFC_MI_con_2021.csv
itr_cia_aberta_DFC_MI_ind_2021.csv
itr_cia_aberta_DMPL_con_2021.csv
itr_cia_aberta_DMPL_ind_2021.csv
itr_cia_aberta_DRE_con_2021.csv
itr_cia_aberta_DRE_ind_2021.csv
itr_cia_aberta_DVA_con_2021.csv
itr_cia_aberta_DVA_ind_2021.csv


In [7]:
FULL_COLUMNS = [
    "CNPJ_CIA",
    "CD_CVM",
    "DT_REFER",
    "DT_INI_EXERC",
    "DT_FIM_EXERC",
    "DENOM_CIA",
    "VERSAO",
    "GRUPO_DFP",
    "MOEDA",
    "ESCALA_MOEDA",
    "ORDEM_EXERC",
    "CD_CONTA",
    "DS_CONTA",
    "VL_CONTA",
    "COLUNA_DF",
]

complete_df = complete_df[complete_df["CD_CVM"].isin(companies_cvm_codes)]
complete_df.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,source_file,DT_INI_EXERC,COLUNA_DF
23586,08.402.943/0001-52,2021-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-03-31,1,Ativo Total,13351642.0,S,itr_cia_aberta_BPA_con_2021.csv,NaN,NaN
23587,08.402.943/0001-52,2021-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-03-31,1.01,Ativo Circulante,8169907.0,S,itr_cia_aberta_BPA_con_2021.csv,NaN,NaN
23588,08.402.943/0001-52,2021-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-03-31,1.01.01,Caixa e Equivalentes de Caixa,2349763.0,S,itr_cia_aberta_BPA_con_2021.csv,NaN,NaN
23589,08.402.943/0001-52,2021-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-03-31,1.01.02,Aplicações Financeiras,404478.0,S,itr_cia_aberta_BPA_con_2021.csv,NaN,NaN
23590,08.402.943/0001-52,2021-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2021-03-31,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,404478.0,S,itr_cia_aberta_BPA_con_2021.csv,NaN,NaN


In [21]:
complete_df.tail()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,source_file,DT_INI_EXERC,COLUNA_DF
88262,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04,Remuneração de Capitais Próprios,449368.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88263,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.01,Juros sobre o Capital Próprio,518118.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88264,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.02,Dividendos,0.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88265,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.03,Lucros Retidos / Prejuízo do Período,-68750.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88266,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.05,Outros,0.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN


In [2]:
from peers_comparison.itr_data_retriever import ITRDataRetriever

itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)

itr_retriever.itr_original_data

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
23586,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1,Ativo Total,13351642.0,None,itr_cia_aberta_BPA_con_2021.csv
23587,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01,Ativo Circulante,8169907.0,None,itr_cia_aberta_BPA_con_2021.csv
23588,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,2349763.0,None,itr_cia_aberta_BPA_con_2021.csv
23589,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02,Aplicações Financeiras,404478.0,None,itr_cia_aberta_BPA_con_2021.csv
23590,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,404478.0,None,itr_cia_aberta_BPA_con_2021.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89714,92.754.738/0001-62,8133,2021-09-30,2021-01-01,2021-09-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04,Remuneração de Capitais Próprios,217326.0,None,itr_cia_aberta_DVA_ind_2021.csv
89715,92.754.738/0001-62,8133,2021-09-30,2021-01-01,2021-09-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.01,Juros sobre o Capital Próprio,260828.0,None,itr_cia_aberta_DVA_ind_2021.csv
89716,92.754.738/0001-62,8133,2021-09-30,2021-01-01,2021-09-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.02,Dividendos,0.0,None,itr_cia_aberta_DVA_ind_2021.csv
89717,92.754.738/0001-62,8133,2021-09-30,2021-01-01,2021-09-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.03,Lucros Retidos / Prejuízo do Período,-43502.0,None,itr_cia_aberta_DVA_ind_2021.csv


In [3]:
from peers_comparison.dfp_data_retriever import DFPDataRetriever

dfp_retriever = DFPDataRetriever()
dfp_retriever.type_format = "ind"


dfp_retriever.get_dfp_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=["DRE"], first_year=first_year, last_year=last_year
)

dfp_retriever.dfp_original_data

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
7105,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1.295988e+06,None,dfp_cia_aberta_DRE_ind_2021.csv
7106,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-1.009910e+06,None,dfp_cia_aberta_DRE_ind_2021.csv
7107,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.03,Resultado Bruto,2.860780e+05,None,dfp_cia_aberta_DRE_ind_2021.csv
7108,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04,Despesas/Receitas Operacionais,2.134520e+05,None,dfp_cia_aberta_DRE_ind_2021.csv
7109,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.01,Despesas com Vendas,-1.110400e+04,None,dfp_cia_aberta_DRE_ind_2021.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24447,92.754.738/0001-62,8133,2021-12-31,2021-01-01,2021-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,None,dfp_cia_aberta_DRE_ind_2021.csv
24448,92.754.738/0001-62,8133,2021-12-31,2021-01-01,2021-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01,Lucro Básico por Ação,6.647000e-01,None,dfp_cia_aberta_DRE_ind_2021.csv
24449,92.754.738/0001-62,8133,2021-12-31,2021-01-01,2021-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01.01,ON,6.647000e-01,None,dfp_cia_aberta_DRE_ind_2021.csv
24450,92.754.738/0001-62,8133,2021-12-31,2021-01-01,2021-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02,Lucro Diluído por Ação,6.628000e-01,None,dfp_cia_aberta_DRE_ind_2021.csv


In [4]:
itr_df = itr_retriever.itr_original_data.copy()
dfp_df = dfp_retriever.dfp_original_data.copy()

In [5]:
import pandas as pd

dfp_itr_df = pd.concat([itr_df[itr_df["source_file"].str.lower().str.contains("dre_ind")], dfp_df]).reset_index(
    drop=True
)
dfp_itr_df["DT_FIM_EXERC"] = pd.to_datetime(dfp_itr_df["DT_FIM_EXERC"])
dfp_itr_df["DT_INI_EXERC"] = pd.to_datetime(dfp_itr_df["DT_INI_EXERC"])
dfp_itr_df["QUARTER"] = dfp_itr_df["DT_FIM_EXERC"].apply(lambda x: f"Q{x.quarter}")

dfp_itr_df["MY_YEAR"] = dfp_itr_df["DT_REFER"].apply(lambda x: x[:4])
dfp_itr_df["EXERC_PERIOD_DAYS"] = (dfp_itr_df["DT_FIM_EXERC"] - dfp_itr_df["DT_INI_EXERC"]).dt.days
dfp_itr_df["SALDO_DF"] = dfp_itr_df["EXERC_PERIOD_DAYS"].apply(
    lambda x: "Acumulado" if x in range(100, 300) else "Trimestre"
)

dfp_itr_df = dfp_itr_df[dfp_itr_df["GRUPO_DFP"].str.contains("Individual")]


# dfp_itr_df.to_csv('dfp_itr.csv')

In [12]:
dfp_itr_df[["CD_CVM", "DENOM_CIA"]].value_counts().reset_index()

,CD_CVM,DENOM_CIA,count
0,22349,AREZZO INDÚSTRIA E COMÉRCIO S.A.,256
1,8133,LOJAS RENNER S.A.,224
2,10456,ALPARGATAS S.A.,204
3,24848,C&A MODAS S.A.,196
4,21440,VESTE S.A. ESTILO,185
5,4146,KARSTEN S.A.,180
6,4669,GUARARAPES CONFECCOES S.A.,180
7,22055,MARISA LOJAS S.A.,177
8,25208,TRACK & FIELD CO S.A.,175
9,26506,LUPO S.A.,171


In [14]:
# dfp_itr_df[dfp_itr_df['SALDO_DF']=='Trimestre'][['CNPJ_CIA', 'CD_CVM', 'DENOM_CIA', 'MY_YEAR', 'GRUPO_DFP'
# , 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF', 'QUARTER', 'VL_CONTA']].sort_values(by=['CNPJ_CIA','CD_CONTA','QUARTER'])


loja_exemplo = dfp_itr_df[
    (dfp_itr_df["CD_CVM"] == 22349)
    & (dfp_itr_df["GRUPO_DFP"] == "DF Individual - Demonstração do Resultado")
    & (dfp_itr_df["CD_CONTA"] == "3.01")
    & (dfp_itr_df["GRUPO_DFP"].str.contains("Individual"))
].copy()
loja_exemplo

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,MY_YEAR,EXERC_PERIOD_DAYS,SALDO_DF
570,16.590.234/0001-76,22349,2021-03-31,2021-01-01,2021-03-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,327292.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021,89,Trimestre
602,16.590.234/0001-76,22349,2021-06-30,2021-01-01,2021-06-30,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,629956.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q2,2021,180,Acumulado
603,16.590.234/0001-76,22349,2021-06-30,2021-04-01,2021-06-30,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,302664.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q2,2021,90,Trimestre
666,16.590.234/0001-76,22349,2021-09-30,2021-01-01,2021-09-30,AREZZO INDÚSTRIA E COMÉRCIO S.A.,2,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1099549.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q3,2021,272,Acumulado
667,16.590.234/0001-76,22349,2021-09-30,2021-07-01,2021-09-30,AREZZO INDÚSTRIA E COMÉRCIO S.A.,2,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,469593.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q3,2021,91,Trimestre
730,16.590.234/0001-76,22349,2021-12-31,2021-01-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1720458.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,364,Trimestre
731,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,620909.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
2172,16.590.234/0001-76,22349,2021-12-31,2021-01-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,2,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1720458.0,None,dfp_cia_aberta_DRE_ind_2021.csv,Q4,2021,364,Trimestre


In [25]:
dfp_itr_df[(dfp_itr_df["QUARTER"] == "Q4") & (dfp_itr_df["EXERC_PERIOD_DAYS"] < 330)]
# [dfp_itr_df['EXERC_PERIOD_DAYS'] > 330]
# [['CD_CVM', 'CD_CONTA', 'VL_CONTA']]

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,MY_YEAR,EXERC_PERIOD_DAYS,SALDO_DF
731,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,620909.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
733,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-411403.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
735,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.03,Resultado Bruto,209506.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
737,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04,Despesas/Receitas Operacionais,-104385.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
739,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.01,Despesas com Vendas,-108432.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
741,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.02,Despesas Gerais e Administrativas,-56089.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
743,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.03,Perdas pela Não Recuperabilidade de Ativos,0.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
745,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.04,Outras Receitas Operacionais,1633.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
747,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.05,Outras Despesas Operacionais,-2230.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre
749,16.590.234/0001-76,22349,2021-12-31,2021-10-01,2021-12-31,AREZZO INDÚSTRIA E COMÉRCIO S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.06,Resultado de Equivalência Patrimonial,60733.0000,None,itr_cia_aberta_DRE_ind_2021.csv,Q4,2021,91,Trimestre


1720458.0

In [7]:
def update_vl_conta(row):
    unit_value = 1000 if row["ESCALA_MOEDA"] == "MIL" else 1
    vl_conta_aux = row["VL_CONTA"] - row["VL_CONTA_AUX"] if row["QUARTER"] == "Q4" else row["VL_CONTA"]

    return unit_value * vl_conta_aux


dfp_itr_df_tri = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Trimestre"].copy()
dfp_itr_df_acu = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Acumulado"].rename(columns={"VL_CONTA": "VL_CONTA_AUX"}).copy()

dfp_itr_df_tri = dfp_itr_df_tri.sort_values(by=["CNPJ_CIA", "CD_CONTA", "QUARTER"]).reset_index(drop=True)
dfp_itr_df_tri = dfp_itr_df_tri.merge(
    dfp_itr_df_acu[dfp_itr_df_acu["QUARTER"] == "Q3"][["MY_YEAR", "CD_CVM", "CD_CONTA", "VL_CONTA_AUX"]],
    how="left",
    on=["MY_YEAR", "CD_CVM", "CD_CONTA"],
)


dfp_itr_df_tri["VL_CONTA"] = dfp_itr_df_tri[["ESCALA_MOEDA", "VL_CONTA", "VL_CONTA_AUX", "QUARTER"]].apply(
    lambda x: update_vl_conta(x), axis=1
)

dfp_itr_df_tri.drop(["EXERC_PERIOD_DAYS", "SALDO_DF", "VL_CONTA_AUX"], axis=1, inplace=True)

dfp_itr_df_tri

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,MY_YEAR
0,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,249837000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021
1,08.402.943/0001-52,4669,2021-06-30,2021-04-01,2021-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,221620000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q2,2021
2,08.402.943/0001-52,4669,2021-09-30,2021-07-01,2021-09-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,393149000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q3,2021
3,08.402.943/0001-52,4669,2021-12-31,2021-01-01,2021-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,431382000.0,None,dfp_cia_aberta_DRE_ind_2021.csv,Q4,2021
4,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-205376000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,92.754.738/0001-62,8133,2021-03-31,2021-01-01,2021-03-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,-185.9,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021
1662,92.754.738/0001-62,8133,2021-03-31,2021-03-01,2021-03-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,0.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021
1663,92.754.738/0001-62,8133,2021-06-30,2021-04-01,2021-06-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,222.7,None,itr_cia_aberta_DRE_ind_2021.csv,Q2,2021
1664,92.754.738/0001-62,8133,2021-09-30,2021-07-01,2021-09-30,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,173.9,None,itr_cia_aberta_DRE_ind_2021.csv,Q3,2021


In [8]:
import peers_comparison.config as config
import numpy as np

ITR_COLUMNS = ["MY_YEAR", "QUARTER", "DT_FIM_EXERC", "DENOM_CIA", "CD_CONTA", "DS_CONTA", "VL_CONTA"]

dfp_itr_df_tri = dfp_itr_df_tri[dfp_itr_df_tri["CD_CONTA"].isin(config.ACCOUNTS)][ITR_COLUMNS]

dfp_itr_df_tri["remove"] = np.where(
    dfp_itr_df_tri["CD_CONTA"].isin(config.INCOMPATIBLE_ACCOUNTS["accounts"]),
    np.where(dfp_itr_df_tri["DS_CONTA"].str.lower().str.contains(config.INCOMPATIBLE_ACCOUNTS["ds_account"]), 0, 1),
    0,
)

dfp_itr_df_tri = (
    dfp_itr_df_tri[dfp_itr_df_tri["remove"] == 0][["MY_YEAR", "QUARTER", "DENOM_CIA", "VL_CONTA", "CD_CONTA"]]
    .replace({"CD_CONTA": config.CD_CONTA_MAP_DICT})
    .replace({"DENOM_CIA": config.UPDATE_COMPANY_NAME})
)

dfp_itr_df_tri

,MY_YEAR,QUARTER,DENOM_CIA,VL_CONTA,CD_CONTA
0,2021,Q1,GUARARAPES CONFECCOES S.A.,249837000.0,receita_de_venda_de_bens_servicos
1,2021,Q2,GUARARAPES CONFECCOES S.A.,221620000.0,receita_de_venda_de_bens_servicos
2,2021,Q3,GUARARAPES CONFECCOES S.A.,393149000.0,receita_de_venda_de_bens_servicos
3,2021,Q4,GUARARAPES CONFECCOES S.A.,431382000.0,receita_de_venda_de_bens_servicos
4,2021,Q1,GUARARAPES CONFECCOES S.A.,-205376000.0,custo_de_bens_servicos
...,...,...,...,...,...
1576,2021,Q1,LOJAS RENNER S.A.,-189068000.0,ebit
1577,2021,Q1,LOJAS RENNER S.A.,0.0,ebit
1578,2021,Q2,LOJAS RENNER S.A.,260701000.0,ebit
1579,2021,Q3,LOJAS RENNER S.A.,181585000.0,ebit


In [9]:
dfp_itr_df_tri.pivot(
    index=["MY_YEAR", "QUARTER", "DENOM_CIA"], columns="CD_CONTA", values="VL_CONTA"
).reset_index().fillna(0)

ValueError: Index contains duplicate entries, cannot reshape

In [2]:
from peers_comparison.itr_data_retriever import ITRDataRetriever


itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)
itr_retriever.pivot_and_clean()

itr_retriever.df_pivot

['bpa_con', 'bpa_ind', 'bpp_con', 'bpp_ind', 'dfc_md_con', 'dfc_md_ind', 'dfc_mi_con', 'dfc_mi_ind', 'dmpl_con', 'dmpl_ind', 'dre_con', 'dre_ind', 'dva_con', 'dva_ind']
['itr_cia_aberta_BPA_con_2021.csv', 'itr_cia_aberta_BPA_ind_2021.csv', 'itr_cia_aberta_BPP_con_2021.csv', 'itr_cia_aberta_BPP_ind_2021.csv', 'itr_cia_aberta_DFC_MD_con_2021.csv', 'itr_cia_aberta_DFC_MD_ind_2021.csv', 'itr_cia_aberta_DFC_MI_con_2021.csv', 'itr_cia_aberta_DFC_MI_ind_2021.csv', 'itr_cia_aberta_DMPL_con_2021.csv', 'itr_cia_aberta_DMPL_ind_2021.csv', 'itr_cia_aberta_DRE_con_2021.csv', 'itr_cia_aberta_DRE_ind_2021.csv', 'itr_cia_aberta_DVA_con_2021.csv', 'itr_cia_aberta_DVA_ind_2021.csv']
['itr_cia_aberta_BPA_con_2022.csv', 'itr_cia_aberta_BPA_ind_2022.csv', 'itr_cia_aberta_BPP_con_2022.csv', 'itr_cia_aberta_BPP_ind_2022.csv', 'itr_cia_aberta_DFC_MD_con_2022.csv', 'itr_cia_aberta_DFC_MD_ind_2022.csv', 'itr_cia_aberta_DFC_MI_con_2022.csv', 'itr_cia_aberta_DFC_MI_ind_2022.csv', 'itr_cia_aberta_DMPL_con_2022.csv

ValueError: Index contains duplicate entries, cannot reshape

In [7]:
import pandas as pd
import peers_comparison.config as config
import numpy as np


# def update_vl_conta(row):
#     unit_value = 1000 if row["ESCALA_MOEDA"] == "MIL" else 1
#     vl_conta_aux = row["VL_CONTA"] - row["VL_CONTA_AUX"] if row["QUARTER"] == "Q4" else row["VL_CONTA"]

#     return unit_value * vl_conta_aux


itr_df = itr_retriever.itr_original_data.copy()
dfp_df = itr_retriever.dfp_original_data.copy()

dfp_itr_df = pd.concat([itr_df[itr_df["source_file"].str.lower().str.contains("dre_ind")], dfp_df]).reset_index(
    drop=True
)
dfp_itr_df["DT_FIM_EXERC"] = pd.to_datetime(dfp_itr_df["DT_FIM_EXERC"])
dfp_itr_df["DT_INI_EXERC"] = pd.to_datetime(dfp_itr_df["DT_INI_EXERC"])
dfp_itr_df["QUARTER"] = dfp_itr_df["DT_FIM_EXERC"].apply(lambda x: f"Q{x.quarter}")

dfp_itr_df["YEAR"] = dfp_itr_df["DT_REFER"].apply(lambda x: x[:4])
dfp_itr_df["EXERC_PERIOD_DAYS"] = (dfp_itr_df["DT_FIM_EXERC"] - dfp_itr_df["DT_INI_EXERC"]).dt.days
dfp_itr_df["SALDO_DF"] = dfp_itr_df["EXERC_PERIOD_DAYS"].apply(
    lambda x: "Acumulado" if x in range(100, 300) else "Trimestre"
)

dfp_itr_df = dfp_itr_df[dfp_itr_df["GRUPO_DFP"].str.contains("Individual")]

dfp_itr_df_tri = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Trimestre"].copy()
dfp_itr_df_acu = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Acumulado"].rename(columns={"VL_CONTA": "VL_CONTA_AUX"}).copy()

dfp_itr_df_tri = dfp_itr_df_tri.sort_values(by=["CNPJ_CIA", "CD_CONTA", "QUARTER"]).reset_index(drop=True)
dfp_itr_df_tri = dfp_itr_df_tri.merge(
    dfp_itr_df_acu[dfp_itr_df_acu["QUARTER"] == "Q3"][["YEAR", "CD_CVM", "CD_CONTA", "VL_CONTA_AUX"]],
    how="left",
    on=["YEAR", "CD_CVM", "CD_CONTA"],
)

dfp_itr_df_tri["VL_CONTA"] = dfp_itr_df_tri[["ESCALA_MOEDA", "VL_CONTA", "VL_CONTA_AUX", "QUARTER"]].apply(
    lambda x: update_vl_conta(x), axis=1
)

dfp_itr_df_tri.drop(["EXERC_PERIOD_DAYS", "SALDO_DF", "VL_CONTA_AUX"], axis=1, inplace=True)

# dfp_itr_df_tri = dfp_itr_df_tri[dfp_itr_df_tri["CD_CONTA"].isin(config.ACCOUNTS)][config.ITR_COLUMNS]

# dfp_itr_df_tri["remove"] = np.where(
#     dfp_itr_df_tri["CD_CONTA"].isin(config.INCOMPATIBLE_ACCOUNTS["accounts"]),
#     np.where(
#         dfp_itr_df_tri["DS_CONTA"].str.lower().str.contains(config.INCOMPATIBLE_ACCOUNTS["ds_account"]), 0, 1
#     ),
#     0,
# )

# dfp_itr_df_tri = (
#     dfp_itr_df_tri[dfp_itr_df_tri["remove"] == 0][["YEAR", "QUARTER", "DENOM_CIA", "VL_CONTA", "CD_CONTA"]]
#     .replace({"CD_CONTA": config.CD_CONTA_MAP_DICT})
#     .replace({"DENOM_CIA": config.UPDATE_COMPANY_NAME})
# )

dfp_itr_df_tri.head()

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,YEAR
0,08.402.943/0001-52,4669,2021-03-31,2021-01-01,2021-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,249837000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q1,2021
1,08.402.943/0001-52,4669,2022-03-31,2022-01-01,2022-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,326419000.0,None,itr_cia_aberta_DRE_ind_2022.csv,Q1,2022
2,08.402.943/0001-52,4669,2021-06-30,2021-04-01,2021-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,221620000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q2,2021
3,08.402.943/0001-52,4669,2022-06-30,2022-04-01,2022-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,366060000.0,None,itr_cia_aberta_DRE_ind_2022.csv,Q2,2022
4,08.402.943/0001-52,4669,2021-09-30,2021-07-01,2021-09-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,393149000.0,None,itr_cia_aberta_DRE_ind_2021.csv,Q3,2021


In [4]:
# df_pivot = (
#     dfp_itr_df_tri.pivot(index=["YEAR", "QUARTER", "DENOM_CIA"], columns="CD_CONTA", values="VL_CONTA")
#     .reset_index()
#     .fillna(0)
# )


count_df = dfp_itr_df_tri[["YEAR", "QUARTER", "DENOM_CIA", "CD_CONTA"]].value_counts().reset_index().copy()

count_df[count_df["count"] > 1]

,YEAR,QUARTER,DENOM_CIA,CD_CONTA,count
0,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,outras_despesas_operacionais,3
1,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,ebit,3
2,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,resultado_bruto,3
3,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,perdas_por_nao_recuperabilidade_de_ativos,3
4,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,outras_receitas_operacionais,3
5,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,receita_de_venda_de_bens_servicos,3
6,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,despesas_receitas_operacionais_totais,3
7,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,despesas_gerais_e_administrativas,3
8,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,despesas_de_vendas,3
9,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,custo_de_bens_servicos,3


In [19]:
# dfp_itr_df_tri[(dfp_itr_df_tri['QUARTER']=='Q4') & (dfp_itr_df_tri['DENOM_CIA']==
# 'AREZZO INDÚSTRIA E COMÉRCIO S.A.')].sort_values(by='CD_CONTA')

# config.ACCOUNTS

# dfp_df

check_df = (
    dfp_itr_df_tri.groupby(["YEAR", "QUARTER", "DENOM_CIA", "CD_CONTA"])["VERSAO"]
    .agg(["count", "max"])
    .reset_index()
    .copy()
)

check_df[(check_df["max"] > 1) & (check_df["count"] > 1)]

,YEAR,QUARTER,DENOM_CIA,CD_CONTA,count,max
1210,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.01,3,2
1211,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.02,3,2
1212,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.03,3,2
1213,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04,3,2
1214,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.01,3,2
1215,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.02,3,2
1216,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.03,3,2
1217,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.04,3,2
1218,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.05,3,2
1219,2021,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,3.04.06,3,2


In [5]:
from peers_comparison.cvm_data_retriever import CVMDataRetriever
import peers_comparison.config as config

cvm_retriever = CVMDataRetriever(type_format="ind")
cvm_retriever.url = config.DFP_URL

files_filter = cvm_retriever.files_filter(first_year=2021, last_year=2022)

cvm_retriever.get_content(files_filter=files_filter)

cvm_retriever.files_list

['dfp_cia_aberta_2021.zip', 'dfp_cia_aberta_2022.zip']

In [11]:
import pandas
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "projetoomni"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = "cvm_peers_reports.itr_report"

pandas_gbq.to_gbq(itr_retriever.df_pivot, table_id, project_id=project_id)

/Users/gustavotronchin/.virtualenvs/fpa_peers_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


GenericGBQException: Reason: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/projetoomni/datasets/cvm_peers_reports/tables?prettyPrint=false: Invalid field name "Custo de Bens/Serviços". Fields must contain the allowed characters, and be at most 300 characters long. For allowed characters, please refer to https://cloud.google.com/bigquery/docs/schemas#column_names

# Melhorias (2024-04-16)

In [6]:
companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2019
last_year = 2019

In [7]:
from peers_comparison.itr_data_retriever import ITRDataRetriever

itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)

itr_retriever.itr_original_data.head()

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
19568,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1,Ativo Total,11119721.0,None,itr_cia_aberta_BPA_con_2019.csv
19569,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01,Ativo Circulante,6002693.0,None,itr_cia_aberta_BPA_con_2019.csv
19570,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,460180.0,None,itr_cia_aberta_BPA_con_2019.csv
19571,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02,Aplicações Financeiras,401834.0,None,itr_cia_aberta_BPA_con_2019.csv
19572,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo,401834.0,None,itr_cia_aberta_BPA_con_2019.csv


In [8]:
original_itr_df = itr_retriever.itr_original_data.copy()
original_itr_df = original_itr_df[original_itr_df["source_file"].str.lower().str.contains("dre_ind")]

original_itr_df.head()

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
33775,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,227419.0,None,itr_cia_aberta_DRE_ind_2019.csv
33776,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-196567.0,None,itr_cia_aberta_DRE_ind_2019.csv
33777,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.03,Resultado Bruto,30852.0,None,itr_cia_aberta_DRE_ind_2019.csv
33778,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04,Despesas/Receitas Operacionais,959.0,None,itr_cia_aberta_DRE_ind_2019.csv
33779,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.01,Despesas com Vendas,-2365.0,None,itr_cia_aberta_DRE_ind_2019.csv


In [26]:
import pandas as pd


def get_saldo_type(row):
    if row["EXERC_PERIOD_DAYS"] in range(80, 100):
        return "Trimestre"
    elif "dfp" in row["source_file"] and row["EXERC_PERIOD_DAYS"] > 300:
        return "Trimestre"
    else:
        return "Acumulado"


def add_aux_columns(original_df):
    df = original_df.copy()

    df["DT_FIM_EXERC"] = pd.to_datetime(df["DT_FIM_EXERC"])
    df["DT_INI_EXERC"] = pd.to_datetime(df["DT_INI_EXERC"])
    df["QUARTER"] = df["DT_FIM_EXERC"].apply(lambda x: f"Q{x.quarter}")

    df["YEAR"] = df["DT_REFER"].apply(lambda x: x[:4])
    df["EXERC_PERIOD_DAYS"] = (df["DT_FIM_EXERC"] - df["DT_INI_EXERC"]).dt.days
    df["SALDO_DF"] = df[["EXERC_PERIOD_DAYS", "source_file"]].apply(lambda x: get_saldo_type(x), axis=1)

    df = df[df["GRUPO_DFP"].str.contains("Individual")]

    return df


def identify_dfp_need(df):
    df_pivot = df[["YEAR", "QUARTER", "CD_CVM", "DENOM_CIA", "CD_CONTA"]].value_counts().reset_index().copy()

    df_pivot = (
        df_pivot.pivot(index=["YEAR", "CD_CVM", "DENOM_CIA", "CD_CONTA"], columns="QUARTER", values="count")
        .reset_index()
        .copy()
    )

    if "Q4" in df_pivot.columns:
        df_pivot = df_pivot[df_pivot["Q4"].isna()][["YEAR", "CD_CVM"]].drop_duplicates()
    else:
        df_pivot = df_pivot[["YEAR", "CD_CVM"]].drop_duplicates()

    return df_pivot.groupby("YEAR")["CD_CVM"].apply(list).reset_index()


# def update_vl_conta(row):
#     unit_value = 1000 if row["ESCALA_MOEDA"] == "MIL" else 1
#     vl_conta_aux = row["VL_CONTA"] - row["VL_CONTA_AUX"] if row["QUARTER"] == "Q4" else row["VL_CONTA"]

#     return unit_value * vl_conta_aux

In [27]:
complete_itr_df = add_aux_columns(original_itr_df)
itr_df_tri = complete_itr_df[complete_itr_df["SALDO_DF"] == "Trimestre"].copy()
itr_df_tri.loc[:, "VL_CONTA_AUX"] = 0

dfp_needed_df = identify_dfp_need(itr_df_tri)

dfp_needed_df

,YEAR,CD_CVM
0,2019,"[4146, 4669, 8133, 10456, 21440, 22055, 22349,..."


In [28]:
from peers_comparison.dfp_data_retriever import DFPDataRetriever

dfp_retriever = DFPDataRetriever()
dfp_retriever.type_format = "ind"

original_dfp_df = pd.DataFrame()

for i, row in dfp_needed_df.iterrows():
    year = int(row["YEAR"])
    dfp_retriever.get_dfp_data(companies_cvm_codes=row["CD_CVM"], type_docs=["DRE"], first_year=year, last_year=year)

    original_dfp_df = pd.concat([original_dfp_df, dfp_retriever.dfp_original_data])

In [29]:
dfp_df = add_aux_columns(original_dfp_df)
dfp_df = dfp_df[dfp_df["SALDO_DF"] == "Trimestre"].copy()

dfp_df

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,YEAR,EXERC_PERIOD_DAYS,SALDO_DF
6861,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1.192138e+06,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
6862,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-9.577740e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
6863,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.03,Resultado Bruto,2.343640e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
6864,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04,Despesas/Receitas Operacionais,3.223190e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
6865,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.01,Despesas com Vendas,-1.270900e+04,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21375,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
21376,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01,Lucro Básico por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
21377,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01.01,ON,1.427600e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre
21378,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02,Lucro Diluído por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre


In [30]:
itr_df_acu = (
    complete_itr_df[(complete_itr_df["SALDO_DF"] == "Acumulado") & (complete_itr_df["QUARTER"] == "Q3")]
    .rename(columns={"VL_CONTA": "VL_CONTA_AUX"})
    .copy()
)

q4_df = dfp_df.merge(itr_df_acu[["YEAR", "CD_CVM", "CD_CONTA", "VL_CONTA_AUX"]], on=["YEAR", "CD_CVM", "CD_CONTA"])

q4_df

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,...,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,YEAR,EXERC_PERIOD_DAYS,SALDO_DF,VL_CONTA_AUX
0,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.01,Receita de Venda de Bens e/ou Serviços,1.192138e+06,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,840384.0000
1,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.02,Custo dos Bens e/ou Serviços Vendidos,-9.577740e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,-684650.0000
2,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.03,Resultado Bruto,2.343640e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,155734.0000
3,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.04,Despesas/Receitas Operacionais,3.223190e+05,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,33644.0000
4,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.04.01,Despesas com Vendas,-1.270900e+04,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,-9134.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,0.0000
272,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.99.01,Lucro Básico por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,0.0000
273,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.99.01.01,ON,1.427600e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,0.7696
274,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.99.02,Lucro Diluído por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre,0.0000


In [31]:
dfp_itr_df = pd.concat([itr_df_tri, q4_df]).reset_index(drop=True)
dfp_itr_df["VL_CONTA"] = dfp_itr_df[["ESCALA_MOEDA", "VL_CONTA", "VL_CONTA_AUX", "QUARTER"]].apply(
    lambda x: update_vl_conta(x), axis=1
)

dfp_itr_df.head()

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,...,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,YEAR,EXERC_PERIOD_DAYS,SALDO_DF,VL_CONTA_AUX
0,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.01,Receita de Venda de Bens e/ou Serviços,227419000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre,0.0
1,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.02,Custo dos Bens e/ou Serviços Vendidos,-196567000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre,0.0
2,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.03,Resultado Bruto,30852000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre,0.0
3,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.04,Despesas/Receitas Operacionais,959000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre,0.0
4,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,...,3.04.01,Despesas com Vendas,-2365000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre,0.0


In [32]:
import peers_comparison.config as config
import numpy as np

dfp_itr_df.drop(["EXERC_PERIOD_DAYS", "SALDO_DF", "VL_CONTA_AUX"], axis=1, inplace=True)

dfp_itr_df = dfp_itr_df[dfp_itr_df["CD_CONTA"].isin(config.ACCOUNTS)][config.ITR_COLUMNS]

dfp_itr_df["remove"] = np.where(
    dfp_itr_df["CD_CONTA"].isin(config.INCOMPATIBLE_ACCOUNTS["accounts"]),
    np.where(dfp_itr_df["DS_CONTA"].str.lower().str.contains(config.INCOMPATIBLE_ACCOUNTS["ds_account"]), 0, 1),
    0,
)

dfp_itr_df = (
    dfp_itr_df[dfp_itr_df["remove"] == 0][["YEAR", "QUARTER", "DENOM_CIA", "VL_CONTA", "CD_CONTA"]]
    .replace({"CD_CONTA": config.CD_CONTA_MAP_DICT})
    .replace({"DENOM_CIA": config.UPDATE_COMPANY_NAME})
)

df_pivot = (
    dfp_itr_df.pivot(index=["YEAR", "QUARTER", "DENOM_CIA"], columns="CD_CONTA", values="VL_CONTA")
    .reset_index()
    .fillna(0)
)

df_pivot

CD_CONTA,YEAR,QUARTER,DENOM_CIA,custo_de_bens_servicos,despesas_de_vendas,despesas_gerais_e_administrativas,despesas_receitas_operacionais_totais,ebit,outras_despesas_operacionais,outras_receitas_operacionais,perdas_por_nao_recuperabilidade_de_ativos,receita_de_venda_de_bens_servicos,resultado_bruto
0,2019,Q1,ALPARGATAS S.A.,-3.889000e+08,-120018000.0,-43442000.0,-182180000.0,66157000.0,-22949000.0,4888000.0,0.0,6.372370e+08,2.483370e+08
1,2019,Q1,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-1.890500e+08,-41428000.0,-26541000.0,-78877000.0,31442000.0,-847000.0,0.0,0.0,2.993690e+08,1.103190e+08
2,2019,Q1,C&A MODAS S.A.,-5.399210e+08,-402333000.0,-127623000.0,109911000.0,610723000.0,639822000.0,0.0,0.0,1.040733e+09,5.008120e+08
3,2019,Q1,GRUPO SBF S.A.,0.000000e+00,0.0,-2328000.0,-4136000.0,-4136000.0,-369000.0,0.0,0.0,0.000000e+00,0.000000e+00
4,2019,Q1,GUARARAPES CONFECCOES S.A.,-1.965670e+08,-2365000.0,-13779000.0,959000.0,31811000.0,0.0,12178000.0,0.0,2.274190e+08,3.085200e+07
5,2019,Q1,KARSTEN S.A.,-3.617900e+07,-14707000.0,-7067000.0,-22328000.0,7799000.0,-2029000.0,3435000.0,0.0,6.630600e+07,3.012700e+07
6,2019,Q1,LOJAS RENNER S.A.,-6.756690e+08,-486126000.0,-179590000.0,-682390000.0,213207000.0,-56746000.0,29432000.0,-26548000.0,1.571266e+09,8.955970e+08
7,2019,Q1,MARISA LOJAS S.A.,-2.264570e+08,-155327000.0,-35150000.0,-239373000.0,-8002000.0,-79093000.0,0.0,0.0,4.578280e+08,2.313710e+08
8,2019,Q1,VESTE S.A. ESTILO,-7.758400e+07,-118828000.0,-27094000.0,-148771000.0,22922000.0,-610000.0,3422000.0,0.0,2.492770e+08,1.716930e+08
9,2019,Q2,ALPARGATAS S.A.,-3.464170e+08,-119795000.0,-35264000.0,-177232000.0,48796000.0,-117591000.0,82645000.0,0.0,5.724450e+08,2.260280e+08


In [34]:
# df_pivot[(df_pivot['QUARTER']=='Q4') & (dfp_itr_df['CD_CVM']==22349) & (dfp_itr_df['CD_CONTA']=='3.99.02.01')]

df_pivot[df_pivot["DENOM_CIA"] == "AREZZO INDÚSTRIA E COMÉRCIO S.A."]

CD_CONTA,YEAR,QUARTER,DENOM_CIA,custo_de_bens_servicos,despesas_de_vendas,despesas_gerais_e_administrativas,despesas_receitas_operacionais_totais,ebit,outras_despesas_operacionais,outras_receitas_operacionais,perdas_por_nao_recuperabilidade_de_ativos,receita_de_venda_de_bens_servicos,resultado_bruto
1,2019,Q1,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-189050000.0,-41428000.0,-26541000.0,-78877000.0,31442000.0,-847000.0,0.0,0.0,299369000.0,110319000.0
10,2019,Q2,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-186314000.0,-39860000.0,-35945000.0,-66715000.0,39999000.0,13885000.0,0.0,0.0,293028000.0,106714000.0
19,2019,Q3,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-208338000.0,-45490000.0,-28429000.0,-73624000.0,53671000.0,5335000.0,0.0,0.0,335633000.0,127295000.0
28,2019,Q4,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-229963000.0,-56304000.0,-45548000.0,-77666000.0,52412000.0,140000.0,0.0,0.0,360041000.0,130078000.0


# Final test

In [1]:
companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2010
last_year = 2011

In [2]:
from peers_comparison.itr_data_retriever import ITRDataRetriever


itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)
itr_retriever.create_report()

itr_retriever.itr_report

,YEAR,QUARTER,CD_CVM,CNPJ_CIA,DENOM_CIA,CD_CONTA,DESC_CONTA,VL_CONTA
0,2011,Q1,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,3.01,Receita de Venda de Bens/Serviços,203340000.0
1,2011,Q1,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,3.02,Custo de Bens/Serviços,-150495000.0
2,2011,Q1,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,3.03,Resultado Bruto,52845000.0
3,2011,Q1,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,3.04,Despesas/Receitas Operacionais (totais),4322000.0
4,2011,Q1,4669,08.402.943/0001-52,GUARARAPES CONFECCOES S.A.,3.04.01,Despesas de Vendas,-2030000.0
...,...,...,...,...,...,...,...,...
846,2011,Q4,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.02,Despesas Gerais e Administrativas,-70043000.0
847,2011,Q4,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.03,Perdas por não recuperabilidade de Ativos,-36195000.0
848,2011,Q4,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.04,Outras Receitas Operacionais,7787000.0
849,2011,Q4,8133,92.754.738/0001-62,LOJAS RENNER S.A.,3.04.05,Outras Receitas Operacionais,-66028000.0


In [3]:
import pandas
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "projetoomni"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = "cvm_peers_reports.itr_report"

pandas_gbq.to_gbq(itr_retriever.itr_report, table_id, project_id=project_id)

/Users/gustavotronchin/.virtualenvs/fpa_peers_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
df_valid = itr_retriever.df_pivot.copy()

df_valid[df_valid["DENOM_CIA"] == "AREZZO INDÚSTRIA E COMÉRCIO S.A."]

CD_CONTA,YEAR,QUARTER,DENOM_CIA,custo_de_bens_servicos,despesas_de_vendas,despesas_gerais_e_administrativas,despesas_receitas_operacionais_totais,ebit,outras_despesas_operacionais,outras_receitas_operacionais,perdas_por_nao_recuperabilidade_de_ativos,receita_de_venda_de_bens_servicos,resultado_bruto
1,2019,Q1,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-189050000.0,-41428000.0,-26541000.0,-78877000.0,31442000.0,-847000.0,0.0,0.0,299369000.0,110319000.0
10,2019,Q2,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-186314000.0,-39860000.0,-35945000.0,-66715000.0,39999000.0,13885000.0,0.0,0.0,293028000.0,106714000.0
19,2019,Q3,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-208338000.0,-45490000.0,-28429000.0,-73624000.0,53671000.0,5335000.0,0.0,0.0,335633000.0,127295000.0
